In [1]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

#import warnings
#warnings.filterwarnings("ignore")

2025-12-26 08:55:22.881392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766739323.136932      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766739323.210961      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766739323.847104      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766739323.847184      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766739323.847190      55 computation_placer.cc:177] computation placer alr

In [2]:
# squad veri seti üzerinde ince ayar yapılmış bert modeli
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# bert tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name) # önceden eğitilmiş BERT modeline ait tokenizer'ı yükleme

# soru cevaplama görevi için ince ayar yapılmış bert modeli
model = BertForQuestionAnswering.from_pretrained(model_name) #önceden eğitilmiş BERT tabanlı soru-cevap modelini yükleme


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# cevapları tahmin eden fonksiyon
def predict_answer(context, question):

    # metni ve soruyu tokenlara ayirma ve modele uygun hale getirme
    encoding = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)
    # Soru ve bağlamı (context) BERT'in anlayacağı şekilde tokenize eder,
    # PyTorch tensörleri olarak döndürür, maksimum uzunluğu 512 token ile sınırlar
    # ve gerekirse metni kırpar (truncation=True)

    # giriş tensorlerini hazırlama
    input_ids = encoding["input_ids"] # tokenlerin idsi
    attention_mask = encoding["attention_mask"] # hangi tokenlarin dikkate alinacagini belirtir

    # modeli calistirma ve skorları hesaplama
    with torch.no_grad(): # Gradientsiz (eğitim yapmadan) model çalıştırmak için no_grad bloğu kullanılır
        start_scores, end_scores = model(input_ids, attention_mask=attention_mask, return_dict = False)
        # Modeli çalıştırarak cevabın başlangıç ve bitiş pozisyonları için skorları alır
        # return_dict=False: çıktının tuple (demet) olarak dönmesini sağlar

    # en yüksek olasılığa sahip start ve end indekslerini hesaplama [1.2, 2.4, 0.3, 2.5,4.7]
    start_index = torch.argmax(start_scores, dim=1).item() # baslangic indeks
    end_index = torch.argmax(end_scores, dim=1).item() # bitis indeksimiz

    # token id lerini kullanarak cevap metnini elde etme
    answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[0][start_index: end_index + 1])
    # Modelin belirlediği başlangıç ve bitiş indeksleri arasındaki token ID'lerini
    # gerçek BERT token'larına (kelime parçacıklarına) dönüştürür


    # tokenları birleştirme ve okunabilir hale getirme
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    return answer


In [4]:
question = "What is the capital of Turkey?"
context = "The capital city of Turkey is Ankara."

answer = predict_answer(context, question)
print("Answer:", answer)


Answer: ankara


In [5]:
question = '''What is Natural Language Processing?'''
context = ''' Natural Language Processing (NLP) is a field of artificial intelligence that enables computers to understand and process human language.
              It analyzes both written and spoken language to extract meaning, answer questions, and generate appropriate responses.
              Technologies such as translation apps, chatbots, voice assistants, and sentiment analysis systems all rely on NLP.
              Because human language is complex and often ambiguous, interpreting it accurately presents significant challenges.
              Therefore, NLP continues to evolve through the use of machine learning and deep learning techniques. '''


answer = predict_answer(context, question)
print("Answer:", answer)

Answer: a field of artificial intelligence


In [6]:
#ingilizce için
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
#Türkçe için
model_name = "ytu-ce-cosmos/turkish-gpt2-large"   # Türkçe GPT-2 modeli

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
#metin üretmek için fonksiyon
def generate_text(prompt, max_length=150):
    # prompt'u token'lara çeviriyoruz ve PyTorch tensörü olarak döndürüyoruz.
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    outputs = model.generate(
        inputs,                 # modele başlangıç token'larını veriyoruz
        max_length=max_length,  # üretilecek maksimum uzunluk
        temperature=0.8,        # yaratıcılık düzeyi (yüksek → daha yaratıcı)
        top_p=0.9,              # nucleus sampling: %90 olasılık kütlesi içinden seçim
        do_sample=True,         # rastgele örneklemeyi aktif eder
        pad_token_id=tokenizer.eos_token_id  # eksik token varsa onları EOS ile doldur
    )
    # üretilen token'ları tekrar metne çeviriyoruz
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
text = generate_text("Today I am learning NLP ")
print(text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Today I am learning NLP "Consciousness" (Zeka ile Bağlantılı)
Konu: Today I am learning NLP "Consciousness" (Zeka ile Bağlantılı)
07.Aralık.2012, 21:10 #1
Today I am learning NLP "Consciousness" (Zeka ile Bağlantılı)
Zeka ve "Consciousness" kavramlarının, günlük yaşamımızda ne gibi etkileri olabilir?
Zeka ve "Consciousness" kavramlarının günlük yaşamımızda ne gibi etkileri olabilir? Bu kavramların günlük yaşamımızdaki etkileri nelerdir?
Zeka, yaşamın her döneminde bir çok alanda karşılaştığımız bir


In [10]:
#!pip install transformers torch gradio

In [11]:
import gradio as gr

def gradio_generate(prompt):
    return generate_text(prompt)

demo = gr.Interface(
    fn=gradio_generate,
    inputs="text",
    outputs="text",
    title="GPT-2 ile Metin Üretimi Uygulaması",
    description="Bir başlangıç cümlesi girin, GPT-2 devamını sizin için yazsın!"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://eae3d6f4d82714437b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
